In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis
!pip install -U -q PyDrive
!pip install yfinance --upgrade --no-cache-dir

import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras

import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

from google.colab import drive

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter

import datetime
from datetime import datetime, timedelta, date

import yfinance as yf
from datetime import datetime, timedelta, date

#mount's to Google Drive
drive.mount('/content/drive')

#connect's to Google Cloud SDK
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/combined_0.csv')
df = df.sample(frac=1)
df.head()

,symbol,message,datetime,user,message_id,Date,Time,label
199321,V,quot valuewalk frenzied alibaba group holding ...,2014-10-16 23:01:30,343779,28099520,2014-10-16,23:01:30,1
6662,T,ms nok son teva latest ratings stocks,2020-04-17 20:07:28,464077,207073905,2020-04-17,20:07:28,1
19786,T,looks like bad call sure careful levels,2019-06-27 21:32:53,2144172,168990298,2019-06-27,21:32:53,1
20469,T,held 32 5 calls weekend 200 banger money bag m...,2019-06-10 14:01:00,1486934,166926327,2019-06-10,14:01:00,0
43886,T,ranks 102 short volume 18 yesterday open 36 77...,2018-01-18 02:55:10,1245096,109338815,2018-01-18,02:55:10,1


In [ ]:
df.drop(df[df['Date'] <= '2019-07-20'].index, inplace = True)
df['label'].value_counts()

1    27545
0    25205
Name: label, dtype: int64

Here we start the process with regression. 
1. Splitting
2. TF-IDF Vetorize
3. Navis Bayes
4. Report

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], random_state = 42, test_size = 0.1)

In [ ]:
y_train.value_counts()

1    24791
0    22684
Name: label, dtype: int64

In [ ]:
y_test.value_counts()

1    2754
0    2521
Name: label, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(use_idf=True,ngram_range=(2,2))

In [ ]:
#Extracting features of the training data
train_features = tf_idf.fit_transform(X_train.values.astype('U'))

test_features = tf_idf.transform(X_test.values.astype('U'))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],}

model = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)
model.fit(train_features, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]}, verbose=5)

In [ ]:
print('Train Accuracy : %.3f'%model.best_estimator_.score(train_features, y_train))
print('Test Accuracy : %.3f'%model.best_estimator_.score(test_features, y_test))
print('Best Accuracy Through Grid Search : %.3f'%model.best_score_)
print('Best Parameters : ',model.best_params_)

Train Accuracy : 0.921
Test Accuracy : 0.571
Best Accuracy Through Grid Search : 0.568
Best Parameters :  {'alpha': 0.5}


In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_features)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.42      0.48      2521
           1       0.57      0.71      0.63      2754

    accuracy                           0.57      5275
   macro avg       0.57      0.56      0.56      5275
weighted avg       0.57      0.57      0.56      5275

